"categories": [
    {
      "id": 0,
      "name": "esca"
    },
    {
      "id": 1,
      "name": "healthy"
    },
    {
      "id": 2,
      "name": "pvl_esca"
    },
    {
      "id": 3,
      "name": "pvl_healthy"
    },
    {
      "id": 4,
      "name": "pvl_unknown"
    },
    {
      "id": 5,
      "name": "unknown"
    }

In [1]:
import glob
import fileinput

annotations_1="C:/Users/Alessandro Pecora/OneDrive - Politecnico di Torino/progetti/DIVINE/label_studio_model/grape_leaves_detection/leaves_detection_model/aledata/labels/*"
annotations_2="C:/Users/Alessandro Pecora/OneDrive - Politecnico di Torino/progetti/DIVINE/label_studio_model/grape_leaves_detection/leaves_detection_model/fdata/labels/*"

cat_data_1=[
    {
      "id": 0,
      "name": "esca"
    },
    {
      "id": 1,
      "name": "healthy"
    },
    {
      "id": 2,
      "name": "pvl_esca"
    },
    {
      "id": 3,
      "name": "pvl_healthy"
    },
    {
      "id": 4,
      "name": "pvl_unknown"
    },
    {
      "id": 5,
      "name": "unknown"
    }
  ]

cat_data_2=[
    {
      "id": 0,
      "name": "esca"
    },
    {
      "id": 1,
      "name": "healthy"
    },
    {
      "id": 2,
      "name": "leaf"
    },
    {
      "id": 3,
      "name": "pvl_esca"
    },
    {
      "id": 4,
      "name": "pvl_healthy"
    },
    {
      "id": 5,
      "name": "pvl_unknown"
    },
    {
      "id": 6,
      "name": "unknown"
    }
]

TO_LABEL = "0" 
PATH_TO_LABELS = "C:/Users/Alessandro Pecora/OneDrive - Politecnico di Torino/progetti/DIVINE/label_studio_model/grape_leaves_detection/leaves_detection_model/fdata/labels/*"

annotations_list = glob.glob(PATH_TO_LABELS)

for annotation in annotations_list:
  with fileinput.FileInput(annotation, inplace=True) as file:
    for line in file:
      parts = line.split(' ')
      acutal_label=parts[0]
      if int(parts[0])>=2:
        parts[0]=str(int(parts[0])-1)
      new_line = ' '.join(parts)
      print(new_line, end='')


In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import shutil
from collections import Counter

# Function to read the primary label from a label file
def read_primary_label(label_file_path):
    with open(label_file_path, 'r') as file:
        lines=file.readlines()
        labels=[]
        if len(lines)==0:
            return -1
        for line in lines:
            labels.append(line.split(" ")[0])
      
    # Get the most common label in the file
    primary_label = Counter(labels).most_common(1)[0][0]
    return primary_label
# Function to create a mapping of file basename to image and label file paths
def create_file_mapping(images_dir, labels_dir):
    file_mapping = []
    for image_filename in os.listdir(images_dir):
        basename, _ = os.path.splitext(image_filename)
        label_filename = basename + '.txt'  # Change extension if necessary
        label_file_path = os.path.join(labels_dir, label_filename)
        if os.path.isfile(label_file_path):
            primary_label = read_primary_label(label_file_path)
            if primary_label==-1:
                continue
            file_mapping.append({
                'basename': basename,
                'image_path': os.path.join(images_dir, image_filename),
                'label_path': label_file_path,
                'primary_label': primary_label
            })
    return pd.DataFrame(file_mapping)

# Function to perform a stratified split and copy files
def stratified_split_and_copy_files(df, train_size=0.8):
    # Perform a stratified split
    train_df, val_df = train_test_split(
        df,
        train_size=train_size,
        stratify=df['primary_label']
    )

    # Create destination directories if they do not exist
    dirs_to_create = [
        'dataset/images/train', 'dataset/images/val',
        'dataset/labels/train', 'dataset/labels/val'
    ]
    for dir_path in dirs_to_create:
        os.makedirs(dir_path, exist_ok=True)

    # Move the files
    for _, row in train_df.iterrows():
        shutil.copy(row['image_path'], 'dataset/images/train')
        shutil.copy(row['label_path'], 'dataset/labels/train')

    for _, row in val_df.iterrows():
        shutil.copy(row['image_path'], 'dataset/images/val')
        shutil.copy(row['label_path'], 'dataset/labels/val')

# Main execution
images_dir = os.path.abspath('./raw_data_full_class/images')
labels_dir = os.path.abspath('./raw_data_full_class/labels')

file_mapping_df = create_file_mapping(images_dir, labels_dir)
stratified_split_and_copy_files(file_mapping_df)


C:\Users\Alessandro Pecora\AppData\Local\Temp\ipykernel_496\2435606622.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
